In [1]:
import os
import sys
sys.path.append("../scripts")
from AttentionTransformer.Encoder import Encoder
from AttentionTransformer.Decoder import Decoder
import torch as T
import torch.nn as nn
import torch.nn.functional as F
from bert4rec_dataset import Bert4RecDataset
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

In [2]:
data = pd.read_csv("../data/ratings_mapped.csv")

In [3]:
ds = Bert4RecDataset(data_csv=data, group_by_col="userId", data_col="movieId_mapped")

In [4]:
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [5]:
batch = next(iter(dl))

In [6]:
batch

{'source': tensor([[2355, 2250, 2008,   18, 1074,  317,    1, 1825, 1482,    1,    1,  632,
             1,  596,    1,    1,    1,  548,    1,    1,    1,    1,  226,  907,
           744,  991,  962,  987,  959,  912, 2346, 1807, 1058,    1,  661,  696,
          2389,  923,  897, 1504,  950,  279,  463,  910,  692,  522,  927,  998,
          1430, 1423, 2197,  735,    1,  125,    1,  100,    1,  947, 1349,  887,
             1, 1668,    1,  262,    1,  988,   57,  813, 2335, 2290,    1, 1434,
          1715,    1, 1260, 1258, 2269,   22,  652, 2319,  854, 1670, 1077,  931,
           992,  976, 1003,  935, 1233,    1,  818, 1755, 1006, 2318,  936,    1,
             2,    1, 1714,    1,  474, 1436, 1758,  400, 1939,    1,    1,  316,
           952,  604,  688,  967, 1068, 1431, 2315, 1496, 2195, 1439,  827,    1],
         [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

In [7]:
# batch.keys()

In [8]:
# batch['target']

In [9]:
# emb_dim = 256
# pad_id = 0
# num_pos = 120
# seq_len = 120
# vocab_size = 9725
# num_channels = 256
# # item_embedding = nn.Embedding(vocab_size, num_channels)
# encoder = Encoder(source_vocab_size=vocab_size,
#                   emb_dim=emb_dim,
#                   layers=4,
#                   heads=6,
#                   dim_model=emb_dim,
#                   dim_key=emb_dim,
#                   dim_value=emb_dim,
#                   dim_inner=emb_dim * 4,
#                   pad_id=pad_id,
#                   num_pos=num_pos)


In [10]:
from bert4rec_model import RecommendationTransformer

In [11]:
model = RecommendationTransformer(9725)

In [12]:
op = model(batch["source"], batch["source_mask"])

In [13]:
op.size()

torch.Size([2, 9725, 120])

In [14]:
batch["target"].size()

torch.Size([2, 120])

In [21]:
F.cross_entropy(op, batch["target"], reduction="mean", ignore_index=0)

tensor(9.3932, grad_fn=<NllLoss2DBackward>)

In [16]:
ll = F.cross_entropy(op, batch["target"], reduction="none", ignore_index=0)
ll

tensor([[ 9.5376, 10.0730,  9.6971,  9.4314,  8.6683,  9.1577,  8.8027, 10.0505,
          7.8386,  8.2026, 10.2838,  9.2237,  9.1152,  8.4808,  9.1966,  9.9811,
          8.2317,  8.4424,  9.0690,  8.3686,  7.6033,  9.9661,  9.3009,  9.6845,
          9.0508,  9.5767,  8.9899,  9.1760, 10.5162,  8.8692,  9.3964,  9.2952,
         10.2361,  9.2877,  9.0753,  9.9128,  9.2751,  9.6946,  9.0251,  9.6576,
          8.9787,  9.0621,  9.4798,  9.1446,  9.7763, 10.3508,  9.8366,  9.1004,
         10.4889,  8.9109, 10.3292,  9.5615,  9.3868, 10.3629,  8.4814,  8.9862,
          9.6342,  9.0076,  9.9588,  7.7236,  9.9108, 10.2908,  9.3413,  9.2431,
          9.7934,  8.6067,  8.3622,  9.4816,  9.8623,  9.7466,  9.1305,  8.9515,
          9.4594,  9.3584, 10.3656,  9.0729,  9.4459, 10.1082, 10.5504,  9.6931,
          9.3437,  9.1972,  9.5284,  9.8898, 10.0449,  8.9951,  9.8634,  8.5865,
          9.5134,  8.5203,  9.9678,  9.9656,  9.7490,  8.0058,  9.5672, 10.1471,
          9.0505,  9.5431,  

In [18]:
ll = ll * batch["target_mask"]

In [20]:
ll.sum() / (batch["target_mask"].sum() + 1e-8)

tensor(9.3932, grad_fn=<DivBackward0>)

In [37]:
# source_item_embedding = item_embedding(batch['source'])

In [38]:
# source_item_embedding.size()

In [62]:
# bs, hist, channels = source_item_embedding.size()

In [63]:
# new_batch = T.arange(0, hist).unsqueeze(0).repeat(bs, 1)

In [64]:
# new_batch.size()

In [40]:
batch["source"].size()

torch.Size([2, 120])

In [42]:
batch["source_mask"].size()

torch.Size([2, 120])

In [65]:
encoder(batch["source"], batch["source_mask"])

RuntimeError: The size of tensor a (2) must match the size of tensor b (6) at non-singleton dimension 1

In [68]:
batch["source_mask"].size()

torch.Size([2, 120])

In [70]:
batch["source_mask"].unsqueeze(1).size()

torch.Size([2, 1, 120])

In [72]:
unsqueezed_mask = batch["source_mask"].unsqueeze(1)

In [44]:
encoder(batch["source"], None).size()

torch.Size([2, 120, 256])

In [46]:
enc_op = encoder(batch["source"], None)

In [45]:
lin_op = nn.Linear(256, vocab_size)

In [49]:
op = lin_op(enc_op)

In [78]:
op.max(2)[-1]

tensor([[6770, 4348, 3933, 5792, 8173,  300, 8761, 7822, 3382, 5557, 1182, 4353,
         4939, 9600, 3488, 8225, 1897, 6763, 7822,  356, 6763, 7245, 1513, 8613,
         8437, 6242, 3515, 6495,  868, 5968, 8459, 3534, 1174,  486,  760, 4087,
         6540, 5164, 9403, 1110, 8433, 5399,  490, 3669, 3762, 5577, 5899, 6980,
         2470, 3573, 5312, 6638, 7611, 6033, 8747, 6678, 7608, 3226, 5873, 1209,
          771, 8721, 1629, 5306, 8721, 6674, 1170, 2286, 3264, 3092, 5996, 1629,
         2377, 7501, 7819, 4935, 7472, 1629, 6129, 1548, 9671, 8468, 7030, 3896,
         5455, 7494, 8004, 6018, 8721, 1262, 8721, 1262, 8721, 8721, 8721, 8721,
         8721, 8721, 8721, 9053, 1262, 1262, 1262, 8721, 8721, 8721, 8531, 8721,
         7775, 2665, 9354, 9354, 9354, 3000, 1552, 8721, 8721, 6919, 6720, 6720],
        [8840, 2677, 4353,  345, 7093, 6294, 1110, 6294, 1110, 2739, 1282, 1282,
         1282, 4353, 4353,  947, 4353, 3569, 9025, 9587, 1282, 2665, 9025, 7819,
         2117, 1629, 8721, 

In [77]:
op.max(2)[-1].masked_fill(batch["source_mask"] == 0, 1e-9)

tensor([[6770, 4348, 3933, 5792, 8173,  300, 8761, 7822, 3382, 5557, 1182, 4353,
         4939, 9600, 3488, 8225, 1897, 6763, 7822,  356, 6763, 7245, 1513, 8613,
         8437, 6242, 3515, 6495,  868, 5968, 8459, 3534, 1174,  486,  760, 4087,
         6540, 5164, 9403, 1110, 8433, 5399,  490, 3669, 3762, 5577, 5899, 6980,
         2470, 3573, 5312, 6638, 7611, 6033, 8747, 6678, 7608, 3226, 5873, 1209,
          771, 8721, 1629, 5306, 8721, 6674, 1170, 2286, 3264, 3092, 5996, 1629,
         2377, 7501, 7819, 4935, 7472, 1629, 6129, 1548, 9671, 8468, 7030, 3896,
         5455, 7494, 8004, 6018,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0, 

In [58]:
op.size()

torch.Size([2, 120, 9725])

In [60]:
op.permute(0, 2, 1).size()

torch.Size([2, 9725, 120])

In [50]:
op.size()

torch.Size([2, 120, 9725])

In [53]:
op.max(1)[-1].size()

torch.Size([2, 9725])

In [54]:
op.max(2)[-1].size()

torch.Size([2, 120])

In [56]:
batch["target"].size()

torch.Size([2, 120])

In [61]:
F.cross_entropy(op.permute(0, 2, 1), batch["target"])

tensor(9.5970, grad_fn=<NllLoss2DBackward>)

In [22]:
op.size()

torch.Size([2, 9725, 120])

In [24]:
batch["target"].size()

torch.Size([2, 120])

In [25]:
op.max(1)[-1].size()

torch.Size([2, 120])

In [37]:
def calculate_accuracy(y_pred: T.tensor, y_true: T.tensor, mask: T.tensor):
    """Calculate the accuracy of the predicted sequence

    Args:
        y_pred (T.tensor): Prediction from the model
        y_true (T.tensor): Ground Truth
        mask (T.tensor): Boolean tensor with True for masked tokens and False
        for unmasked

    Returns:
        T.tensor: Prediction Accuracy
    """
    _, prediction = y_pred.max(1)
    print(prediction)
    y_true = T.masked_select(y_true, mask)
    prediction = T.masked_select(prediction, mask)

    print(y_true)
    print(prediction)

    return (y_true == prediction).double().mean()

In [38]:
calculate_accuracy(op, batch["target"], batch["target_mask"] == 1)

tensor([[9058, 8403, 4231, 2069, 6927, 7686, 1123, 7215, 7132, 1123, 1123,  475,
         1687, 7314, 1687, 1123, 7325, 2774,    6, 7325, 2254, 1687, 5759, 5414,
         2747, 4489, 6974, 1024, 1068, 2437, 2694, 8829, 3611, 1195, 4642,  858,
          575,  544, 7029, 9067, 6756, 4095, 9479, 6402, 3440, 2238, 7671, 1937,
         7911, 8559, 7215, 1811, 5308, 3430, 1195, 8559, 1195, 3449, 7216, 4376,
         6455, 9507, 6455, 8849, 6455, 8705, 3173, 2183, 1465, 6536, 6455, 8881,
         9257, 1687, 7768,  544, 9283, 8951, 8639, 6043, 7277, 1157, 1937, 3363,
         3319, 6592,  544, 8639, 4044,  131, 5348, 4254, 3972, 6669,  720, 1123,
          544, 1123, 3400, 1123, 7216, 3493, 3913, 3430, 4745, 1123,  131, 7882,
         8285, 5619, 3151, 7998, 4292, 1490, 2168, 4403, 6373, 6162, 7882,  131],
        [4403, 1620, 1011, 6372, 8969, 8380, 7277,   35, 5275, 7215, 8748, 8748,
         4227, 6372, 4587, 4227, 4587,  980,  980,  980,  306, 4474, 8921, 6389,
         7215, 8921, 2747, 

tensor(0., dtype=torch.float64)

In [29]:
batch["target_mask"] == 1

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, 